In [1]:
import numpy as np
import pandas as pd

In [2]:

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (12,49,51,56,91,126,145,163,165,166,167,168,169,171,173,174,176,177,242,244,246,248,252,255,274,290,291,292,294,295,296,332,344,366,374,376,397,414,440,491,620,634,639,642,643,645,710,713,760,769,810,829,929,954,979,1001,1002,1003,1004,1005,1024,1037,1041,1043,1062,1086,1099,1100,1121,1129,1136,1152,1153,1166,1168,1182,1193,1204,1205,1207,1208,1216,1226,1228,1230,1232,1234) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (11,13,48,50,55,90,125,144,162,163,164,166,168,169,170,171,172,174,176,241,243,245,247,251,254,273,289,291,292,293,295,296,312,313,314,331,343,365,373,375,396,413,439,490,619,623,630,633,638,642,644,669,712,759,768,788,809,828,928,953,978,1000,1001,1002,1003,1004,1023,1036,1040,1042,

## function for preprocessing

In [3]:
def delete80na_col(train):
    over80 = []
    other = []
    for col in list(train.columns.values):
        per = train[col].isnull().sum()/len(train[col])
        if per > 0.4:
#             print (">80")
            over80.append(col)
        else:
            other.append(col)
    return over80, other

def my_fillna(X):
#     acc = 0
    for col in list(X.columns.values):
        X[col].fillna(round(X[col].mean()), inplace = True)
    return X


In [28]:
def preprocess(train, test):
    df_all = pd.concat([train,test])
    category_cols = train.select_dtypes(exclude=[np.number]).columns.tolist()
    for header in category_cols:
#     df[header] = df[header].astype('category').cat.codes
#     test[header] = test[header].astype('category').cat.codes
        df_all[header] = df_all[header].astype('category').cat.codes.astype('int')
        df_all[header] = pd.to_numeric(df_all[header])

        train[header] = train[header].astype('category').cat.codes
        train[header] = pd.to_numeric(train[header])
        test[header] = test[header].astype('category').cat.codes
        test[header] = pd.to_numeric(test[header])
        
        
    over80, other = delete80na_col(train)
    train_delete_over80 = train.drop(over80, axis = 1)
        
        
    my_fillna(train_delete_over80)
    test = test.reindex(columns = train_delete_over80.columns, fill_value = 0)
    return train_delete_over80, test


In [5]:
import preprocess2 as pre2

In [4]:
train_prepro, test_prepro = pre.preprocess(train, test)

In [14]:
from sklearn.model_selection import train_test_split

def split_into_tr_te(train_prepro):
    X = train_prepro.drop(['is_female','train_id'], axis = 1)
    Y = train_prepro['is_female']
    return train_test_split(X, Y, test_size=0.4, random_state=42)


In [15]:
X_train, X_test, y_train, y_test = split_into_tr_te(train_prepro)
# train_prepro

## PCA

In [10]:
# >>> import numpy as np
# >>> from sklearn.decomposition import PCA
# # >>> X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
# >>> pca = PCA(n_components=2)
# >>> pca.fit(X)
# PCA(copy=True, iterated_power='auto', n_components=10, random_state=None,
#   svd_solver='auto', tol=0.0, whiten=False)
# >>> print(pca.explained_variance_ratio_)  
# # [ 0.99244...  0.00755...]
# >>> print(pca.singular_values_)  
# [ 6.30061...  0.54980...]


NameError: name 'X' is not defined

In [9]:
pca.transform(X)

array([[ -31575.99623401,   -4716.54392315],
       [  23320.58814842,   -5160.16423987],
       [ 123884.89012002,   -5071.33601867],
       ..., 
       [ -54751.73652027,   -4729.41072972],
       [ 145436.85943249,   -5120.34091954],
       [ 125312.15528735,   -5036.71675991]])

### try pca with feature selection

In [12]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest

In [16]:


# This dataset is way too high-dimensional. Better do PCA:
pca = PCA(n_components=2)

# Maybe some original features where good, too?
selection = SelectKBest(k=1)

# Build estimator from PCA and Univariate selection:

combined_features = FeatureUnion([("pca", pca), ("univ_select", selection)])

# Use combined features to transform dataset:
X_features = combined_features.fit(X_train, y_train).transform(X_train)

/usr/local/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [ 22  99 106 107 115 120 122 128 195 248 380 382] are constant.
  UserWarning)
/usr/local/lib/python3.6/site-packages/sklearn/feature_selection/univariate_selection.py:114: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


In [41]:
# X_feature = X_trian %*% W
# W = X_train-1 8 X_feature
m_X_train = X_train.as_matrix()
# m_X_feature = X_features.as_matrix()
# type(m_X_train)
from numpy.linalg import pinv
m_X_train_inv = pinv(m_X_train)

W = np.matmul(m_X_train_inv, X_features)
# print (m_X_train_inv.shape)
# print (X_features)
m_X_test = X_test.as_matrix()
m_X_test_pca = np.matmul(m_X_test, W)
# print (type(m_X_test_pca))
X_test_pca = pd.DataFrame(m_X_test_pca)
print (type(X_train))


<class 'pandas.core.frame.DataFrame'>


In [19]:
print (X_features.shape)
print (X_train.shape)

(10953, 3)
(10953, 410)


# Rate category

indicative feature
['DG5_9', 'DG6', 'DL0', 'DL1', 'DL2', 'DL4_22', 'G2P1_11', 'MT1A', 'MT17_6', 'MT18_4', 'MT18_5', 'FF14_3', 'FF14_4', 'FF14_5', 'FF14_6', 'MM38_14', 'MM40_14', 'FL3', 'GN2', 'GN3', 'GN5', 'DG1', 'DG8a', 'DG9a', 'FF7_5']
'MM38_14'
rate('GN2', 2)
rate('FF14_6') 1
rate('FF14_5')  1
rate('FF14_4')  1
rate('FF14_3') 1
rate('MT18_5') 1
rate('MT18_4') 1
rate('GN5') 1 2
decide later
rate('MT17_6') 

In [33]:
train["GN5"].unique()

# function
# train['FF7_5']
def rate(s):
    for value in range(len(train[s].unique())):
        if len(train[train[s] == value]) != 0:
            print (value, sum(train[train[s] == value]["is_female"])/len(train[train[s] == value]))
    
rate('GN5') 


# encoding category

array([ nan,   0.,   2.,   1.,   4.])

# delete columns with over 80 percent NA

## function print cols with na

In [1]:
# def print_na_col(X):
#     for col in list(X.columns.values):
#         if X[col].isna().sum() > 0:
#             print (col)
# print_na_col(train_delete_over80)

# def print_na_header(X):
#     for head in list(X.columns.values):
#         if head.isna():
#             print (head)
# print_na_header(train_delete_over80)


In [2]:
#train_delete_over80['AA6']

# delete columns with text

In [ ]:
# lst = []
# acc = 0
# for i in (train.dtypes):
#     if str(i) == "object":
#         lst.append(acc)
#     acc = acc+1
    
# lst2 = []
# for i in lst:
#     lst2.append(train.dtypes.keys()[i])

## replace nan with average

In [42]:
from sklearn.ensemble import RandomForestClassifier
TARGET = "is_female"

In [43]:
Ntree = 500
clf = RandomForestClassifier(n_estimators=Ntree,random_state=1234)
clf.fit(X_features, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=1234, verbose=0,
            warm_start=False)

In [44]:
y_pred = clf.predict(X_test_pca)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print (cm)

[[2360 1035]
 [ 896 3011]]


In [45]:
y_pred

array([0, 0, 1, ..., 0, 1, 0])

In [47]:
y_prob = clf.predict_proba(X_test_pca)

In [48]:
y_prob

array([[ 0.986,  0.014],
       [ 0.564,  0.436],
       [ 0.488,  0.512],
       ..., 
       [ 0.87 ,  0.13 ],
       [ 0.006,  0.994],
       [ 0.944,  0.056]])

In [49]:
from sklearn import metrics

In [50]:
metrics.roc_auc_score(y_test, y_prob[:,1])

0.79406838599801799

In [25]:
# feature selection
from sklearn.feature_selection import SelectKBest

In [73]:
train[["is_female","MM28"]][:50]
a = train[["is_female","MM31_9"]].loc[train["MM31_9"] == 2.0]
# a.groupby(["MM31_9","is_female"]).value
# a.pivot(index='is_female', columns='MM31_9')
sum(train["is_female"].loc[train["MM31_9"] == 2.0])
len(train["is_female"].loc[train["MM31_9"] == 2.0])

64

In [74]:
X.columns

Index(['AA3', 'AA4', 'AA7', 'AA14', 'AA15', 'DG1', 'DG3', 'DG3A', 'DG4',
       'DG5_1',
       ...
       'LN1A', 'LN1B', 'LN2_1', 'LN2_2', 'LN2_3', 'LN2_4', 'GN2', 'GN3', 'GN4',
       'GN5'],
      dtype='object', length=237)

In [77]:
dic ={}
for feature in zip(X.columns, clf.feature_importances_):
    dic[feature[1]] = feature[0]
sorted(dic.keys())

[0.0,
 2.5308194883818775e-06,
 2.8699922725177057e-06,
 7.7351872073187215e-06,
 9.0457861986758171e-06,
 1.0696567464831093e-05,
 1.1487894791318577e-05,
 1.2154996715974923e-05,
 1.2157281351453624e-05,
 1.3775475974014467e-05,
 1.4368418196152514e-05,
 1.4770180298541072e-05,
 1.8426274781264229e-05,
 1.9450911752970867e-05,
 2.3538961248779397e-05,
 2.4867387551159507e-05,
 2.5598050578228686e-05,
 2.7164529920667378e-05,
 3.2722399162158611e-05,
 3.7451479316742632e-05,
 3.9813480768029011e-05,
 4.1023970347847128e-05,
 4.4705681176936932e-05,
 4.5119014672365827e-05,
 5.02161016285769e-05,
 5.1234798943489909e-05,
 5.4350105813450348e-05,
 5.7963242735490312e-05,
 5.8838588549038493e-05,
 6.2870704630045256e-05,
 6.7792425365157842e-05,
 7.0140194292703917e-05,
 7.180208699541169e-05,
 7.2477467996036947e-05,
 7.7568975843250797e-05,
 8.033077388119964e-05,
 8.0999134108799452e-05,
 9.2041024639323061e-05,
 9.4660920303399763e-05,
 0.00010408703901697832,
 0.00011077250602151041

In [78]:
dic[0.11546906914365694]

'DL0'

In [117]:
lst = []
for i in sorted(dic.keys()):
    if i>0.01:
        lst.append(dic[i])
lst

['DL14',
 'DG5_4',
 'AA15',
 'AA14',
 'DG8a',
 'AA7',
 'DG4',
 'GN2',
 'DG1',
 'MT10',
 'GN3',
 'GN5',
 'MT2',
 'GN4',
 'DG3',
 'FL4',
 'DL1',
 'DG6',
 'DL0']

In [82]:
X_new = train[lst]
Y_new = train.is_female
# test_new = test.reindex[lst]

x_n_train, x_n_test, y_n_train, y_n_test = train_test_split(X_new, Y_new, test_size=0.4, random_state=42)

Ntree = 500
clf = RandomForestClassifier(n_estimators=Ntree,random_state=1234)
clf.fit(x_n_train, y_n_train)

y_pred = clf.predict(x_n_test)

cm = confusion_matrix(y_n_test, y_pred)
print (cm)

y_prob = clf.predict_proba(x_n_test)
metrics.roc_auc_score(y_n_test, y_prob[:,1])

test_n = test.reindex(columns = x_n_test.columns)

y_pred_f1 = clf.predict(test_n)
y_prob = clf.predict_proba(test_n)


In [90]:
y_prob

array([[ 0.004,  0.996],
       [ 0.85 ,  0.15 ],
       [ 0.01 ,  0.99 ],
       ..., 
       [ 0.998,  0.002],
       [ 0.468,  0.532],
       [ 0.24 ,  0.76 ]])

In [91]:
import csv


In [99]:
with open("submit.csv",'w') as f:
    fieldnames = ["test_id","is_female"]
    wri = csv.DictWriter(f, delimiter=',',fieldnames= fieldnames)
    acc = 0
    wri.writeheader()
    for i in y_prob:
        wri.writerow({"test_id":acc, "is_female": i[1]})
        acc = acc+1

In [100]:
t = pd.read_csv("submit.csv")

In [120]:
new_feature = pd.read_csv("new_feature.csv")
new_feature = new_feature["Column Name"].tolist()
# new_feature  

In [122]:
lst = lst + new_feature


In [124]:
newlist = lst

In [127]:
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values="NaN", strategy = "mean", axis = 1)
imp = imp.fit(train[newlist])

In [ ]:
X_new = imp.transform(train[newlist])
Y_new = train.is_female
# test_new = test.reindex[lst]

x_n_train, x_n_test, y_n_train, y_n_test = train_test_split(X_new, Y_new, test_size=0.4, random_state=42)

Ntree = 500
clf = RandomForestClassifier(n_estimators=Ntree,random_state=1234)
clf.fit(x_n_train, y_n_train)

y_pred = clf.predict(x_n_test)

cm = confusion_matrix(y_n_test, y_pred)
print (cm)

y_prob = clf.predict_proba(x_n_test)
print(metrics.roc_auc_score(y_n_test, y_prob[:,1]))

In [135]:
test_n = test.reindex(columns = newlist)
imp = imp.fit(test)
test = imp.transform(test)


y_pred_f1 = clf.predict(test_n)
y_prob = clf.predict_proba(test_n)

ValueError: could not convert string to float: 'My child'

## try emsemble models!!!

In [116]:
from sklearn.model_selection import cross_val_score
>>> from sklearn.ensemble import BaggingClassifier
>>> from sklearn.neighbors import KNeighborsClassifier
>>> bagging = BaggingClassifier(KNeighborsClassifier(),
...                             max_samples=0.5, max_features=0.5)

In [ ]:
scores = cross_val_score(bagging, X, y)